# Baseline - avg on columns

In [ ]:
import pandas as pd
import numpy as np

url='https://drive.google.com/file/d/1-4YpXkd2kIOM5viSRw8g7oOQm8sicciB/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
y_train = pd.read_csv(url, index_col=0)

url='https://drive.google.com/file/d/1-7VK3dNry2-AYnfRsxMWsOKhHHMTN_ZA/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
test_indices = pd.read_csv(url, index_col=0)

In [ ]:
test_size = test_indices.shape[0]
mean_cols = y_train.mean(axis=0)

In [ ]:
result = pd.concat([mean_cols for i in range(test_size)], axis=1).T
result.head()

In [ ]:
test_indices = test_indices.reset_index()
result["year_weeks"] = test_indices["year_weeks"]
final_res = result.set_index("year_weeks")
final_res.head()

In [ ]:
final_res.to_csv('my_data.csv', index=False)

# functions and implementations

In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, MaxAbsScaler

In [ ]:
def one_hot_incode_date(df, col_name):
  new_df = df.copy()
  new_df = new_df.reset_index()
  datetime_serie = pd.to_datetime(new_df[col_name].map(lambda x: x.split("/")[0]))
  new_df['year'] = pd.DatetimeIndex(datetime_serie).year
  new_df['month'] = pd.DatetimeIndex(datetime_serie).month
  # new_df['day'] = pd.DatetimeIndex(datetime_serie).day
  return new_df

def categorize_seasons(df, month_col_name: str):
  new_df = df.copy()
  new_df["season"] = new_df[month_col_name].map(lambda x: 1 if x in [12,1,2] else (2 if x in [3,4,5] else (3 if x in [6,7,8] else 4)))
  return new_df

def one_hot_encode_columns(df,cols: list):
  new_df = df.copy()
  y = OneHotEncoder().fit_transform(new_df[cols]).toarray()
  return pd.concat([new_df, pd.DataFrame(y)], axis=1)

def add_weeks_col(df):
  weeks_list = list(df.index)
  weeks_num = []
  current = 51
  prev_year = weeks_list[0].split("/")[1].split("-")[0]
  for dates in weeks_list:
    weeks_num.append(current)
    dates_list = dates.split("/")
    if int(dates_list[0].split("-")[0]) != int(dates_list[1].split("-")[0]) or int(prev_year) != int(dates_list[0].split("-")[0]):
      current = 0
    else:
      current += 1
    prev_year = dates_list[1].split("-")[0]
  new_df = df.copy()
  new_df["week"] = weeks_num
  return new_df

def sinus(df,lst):
  new_df = df.copy()
  for i in lst:
    m = len(pd.unique(pd.Series(new_df[i])))
    temp = [np.sin((x / m) * np.pi * 2) for x in new_df[i]]
    new_df["new "+ i] = temp
    new_df = new_df.drop(i,axis=1)
  return new_df


def threshold_round_down(y_pred, beta, gamma):
  temp = y_pred.copy(deep=True)
  temp = one_hot_incode_date(temp,'year_weeks')\
        .set_index('year_weeks')
  for i in range(1,13):
     temp1 = temp[temp['month']==i]
     indexes = temp1.index
     for col in y_pred.columns:
       diff = temp1[col].mean() - beta*temp1[col].std()
       for idx in indexes:
         if temp.loc[idx,col] < diff:
            temp.loc[idx,col] /= gamma
  return temp.iloc[:,:-2]

In [ ]:
add_weeks_col(y_train)

In [ ]:
enc_df = one_hot_incode_date(y_train, "year_weeks")
w_seasons_df = categorize_seasons(enc_df, "month")
df_ready_for_reg = one_hot_encode_columns(w_seasons_df, ['month', "season"]).drop("year_weeks", axis=1)
df_ready_for_reg.head()

,other_יהודה ושומרון,other_מחוז הדרום,other_מחוז המרכז,other_מחוז הצפון,other_מחוז חיפה,other_מחוז ירושלים,other_מחוז תל אביב,אבטיחים_other,אבטיחים_יהודה ושומרון,אבטיחים_מחוז הדרום,...,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,3500.0,800.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,780.0,4700.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,2000.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1200.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
year_s = df_ready_for_reg.year
df_ready_for_reg = df_ready_for_reg.drop("year", axis=1)
df_ready_for_reg["year"] = year_s
df_ready_for_reg.head()

,other_יהודה ושומרון,other_מחוז הדרום,other_מחוז המרכז,other_מחוז הצפון,other_מחוז חיפה,other_מחוז ירושלים,other_מחוז תל אביב,אבטיחים_other,אבטיחים_יהודה ושומרון,אבטיחים_מחוז הדרום,...,7,8,9,10,11,12,13,14,15,year
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2009
1,0.0,3500.0,800.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2010
2,0.0,0.0,780.0,4700.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2010
3,0.0,0.0,0.0,2000.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2010
4,0.0,1200.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2010


# **Testing**

## Linear Ridge alpha=63 : Basic (best result of 5486)

In [ ]:
from sklearn import datasets, linear_model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=50,max_depth=4, random_state=0,min_samples_leaf=3)

In [ ]:
linear_model.Ridge(alpha=64,max_iter=1e18, tol=1e-9)

Ridge(alpha=64, max_iter=1e+18, tol=1e-09)

THE RIDGE!

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = linear_model.Ridge(alpha=64,max_iter=1e18, tol=1e-9)


# training set
enc_df = one_hot_incode_date(y_train, "year_weeks")
enc_df = categorize_seasons(enc_df, "month")
enc_df = add_weeks_col(enc_df.set_index("year_weeks"))
enc_df = sinus(enc_df, ["season","month","week"])

# test set
df_test = one_hot_incode_date(test_indices, "year_weeks")
df_test = categorize_seasons(df_test, "month")
df_test = add_weeks_col(df_test.set_index("year_weeks"))
df_test = sinus(df_test, ["season","month","week"])
x_train = enc_df.iloc[:,188:]
y_train = enc_df.iloc[:,:188]
x_test = df_test


regr.fit(x_train,y_train)
y_pred = regr.predict(x_test)
y_pred[y_pred < 0] = 0

final_y_pred = pd.DataFrame(y_pred,index=test_indices.reset_index()["year_weeks"],columns=list(y_train.columns))
final_y_pred = pd.DataFrame(y_pred,index=test_indices.reset_index()["year_weeks"],columns=list(y_train.columns))
final_y_pred.reset_index().to_csv('pred_RF3-50-5.csv', index=False)

In [ ]:
# prepare testing data
df_test = one_hot_incode_date(test_indices, "year_weeks")
df_test = categorize_seasons(df_test, "month")
x_test = one_hot_encode_columns(df_test, ['month',"day", "season"]).drop("year_weeks", axis=1)

year_s = x_test.year
x_test = x_test.drop("year", axis=1)
x_test["year"] = year_s
x_test = x_test.iloc[:,3:]

# Make predictions using the testing set
y_pred = regr.predict(x_test)
y_pred[y_pred < 0] = 0

final_y_pred = pd.DataFrame(y_pred,index=test_indices.reset_index()["year_weeks"],columns=list(y_train.columns))
final_y_pred = pd.DataFrame(y_pred,index=test_indices.reset_index()["year_weeks"],columns=list(y_train.columns))
final_y_pred[final_y_pred <= 1160] = 0
final_y_pred[final_y_pred <= 6900] /= 2
final_y_pred.reset_index().to_csv('pred_ridge_final.csv', index=False)

## Linear Ridge Model extended

In [ ]:
# training set

enc_df = one_hot_incode_date(y_train, "year_weeks")
enc_df = categorize_seasons(enc_df, "month")
enc_df = one_hot_encode_columns(enc_df, ["season","month"])
enc_df = add_weeks_col(enc_df.set_index("year_weeks"))
enc_df = sinus(enc_df, ["season","month","week"])

# test set
df_test = one_hot_incode_date(test_indices, "year_weeks")
df_test = categorize_seasons(df_test, "month")
df_test = one_hot_encode_columns(df_test, ["season","month"])
df_test = add_weeks_col(df_test.set_index("year_weeks"))
df_test = sinus(df_test, ["season","month","week"])
x_train = enc_df.iloc[:,188:]
y_train = enc_df.iloc[:,:188]
x_test = df_test

In [ ]:
normalizer = MinMaxScaler()
normalizer2 = MinMaxScaler()
normalizer2.fit(y_train)
o_y_train = y_train.copy()
y_train = normalizer2.transform(y_train)
normalizer.fit(x_train)
x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=1300,max_depth=4, random_state=0\
                             ,bootstrap=True,min_samples_leaf=3)

regr.fit(x_train,y_train)
y_pred = regr.predict(x_test)
y_pred = normalizer2.inverse_transform(y_pred)
y_pred[y_pred < 0] = 0

final_y_pred = pd.DataFrame(y_pred,index=test_indices.reset_index()["year_weeks"],columns=list(o_y_train.columns))
final_y_pred = pd.DataFrame(y_pred,index=test_indices.reset_index()["year_weeks"],columns=list(o_y_train.columns))
# final_y_pred = threshold_round_down(final_y_pred, beta=2.5,gamma=2.5)
# final_y_pred[final_y_pred <= 6000] /= 3
final_y_pred.reset_index().to_csv(f'pred_RF.csv', index=False)